### Higher Diploma in Science - Data Analytics 2020/21

### Fundamentals of Data Analysis Project

#### Prepared by: Fiona Lee 
#### Due Date - 8 January 2021


<BR>

### Fundamentals of Data Analysis - Project Instructions:

In  this  project  you  must  perform  and  explain  simple  linear  regression  using  Python on the power production dataset available on Moodle.  

The goal is to accurately predict wind turbine power output from wind speed values using the data set as a basis.

Your submission must be in the form of a git repository containing, at a minimum, thefollowing items:

1.  Jupyter notebook that performs simple linear regression on the data set.
2.  In that notebook, an explanation of your regression and an analysis of its accuracy.
3.  Standard items in a git repository such as a README.

To enhance your submission, you might consider comparing simple linear regression to other types of regression on this data set. 

### Import Modules 
***

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame
from sklearn.metrics import r2_score 
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
%matplotlib inline

<BR>

### Import & Check the Dataset
***

#### Import the Dataset

In [ ]:
df = pd.read_csv('Wind-Power.csv')
pd.set_option('max_colwidth',100)

#### Check if characters in the data imported are the same type

In [ ]:
df.dtypes

#### Check if there are any empty cells in the dataframe

In [ ]:
pd.set_option("display.max_rows",500)# Option to display all entries
df.style.highlight_null(null_color='red'); #highlight any null values in red

#Option to print the dataset
#print(df.isnull())  

#Alternative method (https://stackoverflow.com/questions/30447083/python-pandas-return-only-those-rows-which-have-missing-values)
#df[pd.isnull(df).any(axis=1)]

#*Note not printed due to the size of the dateset.  No null values noted*

#### Check for corrupt data

In [ ]:
print ("power shape = ",df.shape)

#### Re-name the columns from the original dataset

In [ ]:
#Re-name the columns from the original dataset
df.rename(columns={'speed':'Speed_Mph'},inplace = True)
df.rename(columns={'power':'Power_KWph'},inplace = True)

<BR>

### Review the Data
***

#### Show the entire dataset sorted by 'Speed_Mph'

In [ ]:
pd.set_option("display.max_rows", None)#Show the entire dataset
pd.set_option("display.precision", 3) #Round the results to 3 decimal places
df.sort_values(by =['Speed_Mph'], inplace=True, ascending=True)#Sort the dataset in ascending order
#https://datatofish.com/sort-pandas-dataframe/
df.style.set_properties(subset = ["Power KWph", "Speed Mph"], **{'text-align': 'center'})
#https://www.kite.com/python/answers/how-to-print-an-entire-pandas-dataframe-in-python
pd.set_option('expand_frame_repr', False) #print dataframe on a single line
#https://stackoverflow.com/questions/39482722/how-to-print-dataframe-on-single-line
#print((df[['Speed_Mph','Power_KWph','Power:Speed','Classification']]).head(10))

#### Option to Print the Dataset

In [ ]:
df.head(5)

<BR>

### Add Classification Columns
***

#### Create a New Column called 'Power:Speed'

In [ ]:
df['Power:Speed'] = (df ['Power_KWph'] / df ['Speed_Mph']) #Add Power/Speed Column
pd.set_option("display.precision", 3)
df.style.set_properties(subset=["Power_KWph", "Speed_Mph"], **{'text-align': 'centre'});
#https://www.codegrepper.com/code-examples/python/how+to+align+column+name+to+center+in+pandas+dataframe

#### Create a New Column called 'Optimum Ratio' based on the 'Power:Speed' Ratio

In [ ]:
def Optimum(row):

    if row['Power:Speed'] ==0 and row['Speed_Mph'] > 8:
        return 'Assumed Out of Order'
    if row['Power:Speed'] ==0 and row['Speed_Mph'] < 8:
        return 'Zero Output - Low Wind Speed (< 8mph)'
    if row['Speed_Mph']== 0 and row['Power_KWph'] == 0:
        return 'Test'
    if row['Power:Speed'] > 0 and row['Power:Speed'] < 6 and row['Speed_Mph'] < 8:
        return 'Sub Optimum Power:Speed - Low Wind Speed (<8 mph)' 
    if row['Power:Speed'] > 0 and row['Power:Speed'] < 6 and row['Speed_Mph'] >= 8:
        return 'Sub Optimum Power:Speed - Wind Speed > 8 mph'     
    if row['Power:Speed'] >= 6 and row['Power:Speed'] <= 8:
        return 'Optimum Output: Power:Speed (6-8)'         
    if row['Power:Speed'] > 8 and row['Speed_Mph'] < 8 :
        return 'Exceptional Output: Wind Speed ( < 8mph)'    
    if row['Power:Speed'] > 8 and row['Speed_Mph'] >= 8:
        return '> Optimum'             
    else:
        return 'null'


#### Populate the 'Optimum_Ratio' Column  

In [ ]:
df['Classification'] = df.apply(lambda row: Optimum(row), axis=1) 

<BR>

### Summarise the Data
***

#### Show basic summary information

In [ ]:
pd.set_option("display.precision", 1)
# https://realpython.com/pandas-groupby/
df.describe()

#### Summarise the Data by Power:Speed Classification

In [ ]:
df.style.set_properties(subset = ['Classification','Power_KWph'], **{'text-align': 'left'})
print(df[['Classification','Power_KWph']].groupby(['Classification']).count(),"\n")

<BR>

### Identify Trends & Remove Outliers:
***

#### Group Results by Power Output Value

In [ ]:
print("")
pd.set_option("display.precision", 2)
# https://realpython.com/pandas-groupby/
df.style.set_properties(subset = ["Power_KWph", "Speed_Mph"], **{'text-align': 'center'});

#Option to print the analysis
#df.groupby(['Power_KWph']).agg(['count'])

#Note - Results not shown due to size - summary of findings below

#### Summary of Findings:

49 Samples had a 'Power Output' of Zero (See below for analysis):

- 14 samples were identified as being 'outliers' i.e. where wind speed was in excess of 8 mph and power output was zero.  These samples were removed as it can be assumed that the turbine was out of order.  These samples were classed as 'Assumed Out of Order'  

- 1 sample was removed where both power output and wind speed were zero.  This was classed as a 'Test' and has been excluded.

- 34 samples were identified where power output was zero at wind speeds less than 8mph.  It has been assumed that this is normal and should be part of the prediction model.

10 Results where Power:Speed Ratio was in excess of 8 at low wind speeds (<8 mph)(See below for analysis):

- These samples could be classed as further  possible 'outliers ' as they are exceptional results and potentially incorrect records.  Without further invesigation I decided not to exclude them for the purposes of this analysis.

#### Analysis of 49 Results where Power Output was Zero

In [ ]:
df.sort_values(by=['Speed_Mph'], inplace=True, ascending=True)#Sort the dataset in descending order

#Option to print the dataset subset where Power_KWph = 0
df[df['Power_KWph'] == 0]


#### Analysis of 10 Possible Outliers (not Excluded)

In [ ]:
#df.loc[(df.Power:Speed > 0) & (df.Speed_Mph < 8)]
df.loc[(df.Power_KWph/df.Speed_Mph > 8) & (df.Speed_Mph < 8)]

### Remove Outlier Data from the Dataframe
***

#### Remove Rows where Power_KWph = 0 and Speed_Mph > 8 or Speed_Mph = 0

In [ ]:
Out_of_Order = df.loc[(df.Power_KWph == 0) & (df.Speed_Mph > 8)] # identify rows where true
df.drop(Out_of_Order.index, inplace=True)#Remove these rows from the existing dataset
Test = df.loc[(df.Power_KWph == 0) & (df.Speed_Mph == 0)] # identify rows where true
df.drop(Test.index, inplace=True) #Remove these rows from the existing dataset
# https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/

<BR>

### Visualise the Data
***

#### Plot the Dataset excluding Outliers

In [ ]:
plt.figure(figsize = (8, 5)) 
plt.title('\nRelationship between Wind Speed and Power Output\n', fontsize = 14)
plt.xticks([0, 2, 4, 6,8,10,12,14,16,18,20,22,24,26], size =14)
plt.yticks([0, 10,20,30,40,50,60,70,80,90,100,110,120], size =14)
plt.grid(True)
plt.rcParams["axes.labelsize"] = 14
sns.lineplot (x=df.Speed_Mph,y=df.Power_KWph)
plt.show()

<BR>

#### Plot the Frequency of Speed and Power

In [ ]:
print ("")
plt.figure(figsize = (10, 7)) 
x = df ["Speed_Mph"] 
plt.hist(x, bins = 20, color = "blue", rwidth = 0.95) 
plt.title("Speed Values Occurences in Wind Turbine Dataset\n", fontsize = 15) 
plt.xlabel("Speed", fontsize = 15) 
plt.ylabel("Frequency", fontsize = 15) 
plt.show()
print ("")
plt.figure(figsize = (10, 7)) 
x = df ["Power_KWph"] 
plt.hist(x, bins = 20, color = "red", rwidth = 0.95) 
plt.title("Power Value Occurences in Wind Turbine Dataset\n", fontsize = 15) 
plt.xlabel("Power", fontsize = 15) 
plt.ylabel("Frequency", fontsize = 15) 
plt.show()

<BR>

#### Plot Speed and Power Results on the Same Axis


In [ ]:
#https://stackoverflow.com/questions/11640243/pandas-plot-multiple-y-axes
print("")
plt.rcParams["axes.labelsize"] = 8
sns.set_style ("whitegrid")
fig, ax = plt.subplots()
rspine = ax.spines ['right']
rspine.set_position (('axes', 1))
ax.set_frame_on (True)
ax.patch.set_visible (True)
ax.grid(True)
fig.subplots_adjust(right = 2)
df.Power_KWph.plot(ax = ax, style = 'b.',label = "Power Produced")
plt.yticks([0,10,20,30,40,50,60,70,80,90,100,110,120],fontsize = 16)
ax.legend(loc = 0)
plt.xlabel("\n Sample Number",fontsize = 16)
plt.ylabel("Power Produced (KWph) \n",fontsize = 16)

# same ax as above since it's automatically added on the right
df.Speed_Mph.plot(ax = ax, style = 'r.', label = "Wind Speed", secondary_y = True)
plt.yticks([0, 2, 4, 6,8,10,12,14,16,18,20,22,24,26],fontsize = 16)
plt.xticks([0,25, 50,75, 100, 125, 150, 175, 200, 225, 250, 275,300,325,350,375,400,425,450,475,500],fontsize = 16)
plt.ylabel("\n Wind Speed (Mph)",fontsize = 16)
plt.title("\n Relationship between the rate of growth of Wind Speed vs. Power Produced by Wind Turbines \n\n" , fontsize = 16)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels)
plt.legend (loc = "center left", frameon = True)
bbox_to_anchor=(10, 500)
plt.show()


<BR>

### Calculate Slope (m) and Co-efficient (c) Values
***

#### Polyfit Method

In [ ]:
#https://github.com/ianmcloughlin/jupyter-teaching-notebooks/blob/master/simple-linear-regression.ipynb

Polyfit = np.polyfit (df.Speed_Mph,df.Power_KWph,1)
print ('\nBest Fit Line Values:\n')
print ("- 'm' (slope) of 'Best Fit Line' is", round(Polyfit[0],6) )
print ("- 'c' (co-efficient) of 'Best Fit Line' is", round(Polyfit[1],6),"\n" )

#### Alternative Manual Method

In [ ]:
# First calculate the means of df.Speed_Mph and df.Power_KWph:
Speed_Avg = np.mean(df.Speed_Mph)
Power_Avg = np.mean(df.Power_KWph)

# Subtract means from df.Speed_Mph and df.Power_KWph:
Speed_Zero = df.Speed_Mph - Speed_Avg
Power_Zero = df.Power_KWph - Power_Avg

# The best m is found by the following calculation:
m = np.sum(Speed_Zero * Power_Zero) / np.sum(Speed_Zero * Speed_Zero)

# Use m from above to calculate the best c:
c = Power_Avg - m * Speed_Avg

print ('\nBest Fit Line Values:\n')
print ("- The slope of the 'Best Fit Line'(m)is",(round(m,6)))
print ("- The value of y when x = 0 of the 'Best Fit' Line (c) is",round(c,6),"\n")

#### Fitting the Model - Trial Lines

In [ ]:
print ("")
plt.rcParams["figure.figsize"] = (28,24)
#plt.figure(figsize = (8, 6)) 
sns.set_style ("whitegrid")
#plt.plot (df2.Speed_Mph,df2.Power_KWph,'y.', label = r"Actual Data")
sns.relplot(x = "Speed_Mph", y = 'Power_KWph', palette="muted",height = 7, data=df)#data excluding power outages
plt.title("Speed vs Power - Trial Lines\n\n", fontsize = 16)
plt.rcParams["axes.labelsize"] = 14
plt.xticks([0, 2, 4, 6,8,10,12,14,16,18,20,22,24,26], fontsize = 14)
plt.yticks([0, 10,20,30,40,50,60,70,80,90,100,110,120], fontsize = 14);

#Generate Trial Data to estimate the 'Best Fit Line'
x = np.arange (0.0,25.0,(1/len(df.Speed_Mph)*25))
plt.plot (x, 5.5 * x + 0.5, 'b-', label = r"Trial Fit 1:5.5x + 0.5")#Trial Fit 1
plt.plot (x, 5 * x + 0.5, 'g-', label = r"Trial Fit 2: 5x + 0.5") #Trial Fit 2
plt.plot (x, 4 * x + 5, 'r-', label = r"Trial Fit 3: 4x + 5")#Trial Fit 3
plt.legend()
plt.show()

#### Fitting the Model - Best Fit Lines

In [ ]:
print ("")
plt.rcParams["figure.figsize"] = (28,24)
sns.set_style ("whitegrid")
sns.relplot(x = "Speed_Mph", y = 'Power_KWph', palette="muted",height = 7, data=df)#use data excluding power outages
plt.title("Speed vs Power - 'Best Fit Line'\n\n", fontsize = 16)
plt.rcParams["axes.labelsize"] = 14
plt.xticks([0, 2, 4, 6,8,10,12,14,16,18,20,22,24,26], fontsize = 14)
plt.yticks([0, 10,20,30,40,50,60,70,80,90,100,110,120], fontsize = 14);

#Use the Polyfit results to generate the 'Best Fit Line' 
y = Polyfit[0] * x + Polyfit[1]
plt.plot (x, y, 'k-', label = r"Actual Best Fit: 5.56x = -19.02")# Plot the best fit line.
plt.legend()
plt.show()

#### Alternative Method of Finding the 'Best Fit' Line -  Scatterplot with a Regression Line:

In [ ]:
#print ("")
plt.rcParams["figure.figsize"] = (12,8);
plt.rcParams["axes.labelsize"] = 14
sns.set_style ("whitegrid");
sns.lmplot(x="Speed_Mph", y='Power_KWph', hue='Classification', palette="deep", height = 7 ,data=df);
plt.title("Speed vs Power\n\n", fontsize = 16);
plt.rcParams["axes.labelsize"] = 16;
plt.xticks([0, 2, 4, 6,8,10,12,14,16,18,20,22,24,26], fontsize = 14);
plt.yticks([0, 10,20,30,40,50,60,70,80,90,100,110,120], fontsize = 14);
plt.show()

<BR>

### Predict Expected Power based on a Given Speed
***

#### Using Numpy Poly1d Method

In [ ]:
Predict = np.poly1d (Polyfit)
#https://data36.com/linear-regression-in-python-numpy-polyfit/

<BR>

#### Insert Value for 'P' (Speed) to Predict Power Output

In [ ]:
#Input the Value of Speed (S) 

S = 6  #P (Wind Speed) is a variable to predict Power Output 

print ("\nThe Predicted Power Output for a Wind Speed of",S,"Mph is",round(Predict(S),2),"KWph\n")

#### Manual Calculation of Power Output

In [ ]:
#y = c + mx
y = (c + 6*m) 
print ("\nThe Predicted Power Output for a Wind Speed of",S,"Mph is",round(y,2),"KWph\n")

#### Alternative Method - Manual method using Polyfit Results:

In [ ]:
y = Polyfit [0] * S + Polyfit [1]
print ("\nThe Predicted Power Output for a Wind Speed of",S,"Mph is",round(y,2),"KWph\n")

<BR>

### Analyse the Accuracy of Trial and Best Fit Lines
***

#### Calculate the 'Cost' of Trial Fit Line

In [ ]:
p = (np.array(df.Power_KWph))
s = (np.array(df.Speed_Mph))
pd.set_option("display.precision", 2) #Round the results to 2 decimal places

In [ ]:
cost = lambda m,c: np.sum([(p[i] - m * s[i] - c)**2 for i in range(len(df.Speed_Mph))])

print("\nTrial Fit 1: Cost with m = %.1f and c = %.1f: %.0f" % (5.5, 0.5, cost(5.5, 0.5)))
print("Trial Fit 2: Cost with m = %.1f and c = %.1f: %.0f" % (5.0,  0.5, cost(5.0,  0.5)))
print("Trial Fit 3: Cost with m = %.1f and c = %.1f: %.0f" % (4.0, 5.0, cost(4.0, 5.0)),"\n")

#### Calculate the 'Cost' of Best Fit Line

In [ ]:
cost = lambda m,c: np.sum([(p[i] - Polyfit[0] * s[i] - Polyfit[1])**2 for i in range(len(df.Speed_Mph))])
print("\nBest Fit: Cost with m = %.2f and c = %.2f: %.0f" % (m, c, cost(m, c)),"\n")

In [ ]:
#Alternative code using m & c as variables:
cost = lambda m,c: np.sum([(p[i] - m * s[i] - c)**2 for i in range(len(df.Speed_Mph))])
print("\nBest Fit: Cost with m = %.2f and c = %.2f: %.0f" % (m, c, cost(m, c)),"\n")

   *Conclusion: The 'Best Fit' line is the closest linear estimation of the dataset*

<BT>

#### Check Results & Create Cost Column in the Dataframe

In [ ]:
#To create data for df['Cost']
Cost = ([(p[i] - m * s[i] - c)**2 for i in range(len(df.Speed_Mph))])

In [ ]:
# Create a 'Cost Column to show Cost by Sample'
df['Cost'] = Cost
pd.set_option("display.precision", 3) #Round the results to 3 decimal places

In [ ]:
#Option to show the dataframe with 'Cost' by sample (only 5 shown as an example for space purposes)
df.head(5)

In [ ]:
print ("\nThe Cost of the Best fit is",((round(np.sum(df['Cost'])))),"\n")

#### Calculate the R2 Value of the Trial Fit Lines

In [ ]:
#https://www.geeksforgeeks.org/python-coefficient-of-determination-r2-score/
a = np.array(df.Power_KWph)

#Trial Fit Line 1:
b1 = np.array(5.5 * x + 0.5)
r2 = r2_score(a,b1)
print("\nThe R2 value of Trial Fit Line 1 is",(round(r2*100)),"%")

#Trial Fit Line 2:
b2 = np.array(5 * x + 0.5)
r2 = r2_score(a,b2)
print("The R2 value of Trial Fit Line 2 is",(round(r2*100)),"%")

#Trial Fit Line 3:
b3 = np.array(4 * x + 5)
r2 = r2_score(a,b3)
print("The R2 value of Trial Fit Line 3 is",(round(r2*100)),"%\n")

#### Calculate the R2 Value of the Best Fit Lines

In [ ]:
#Create and populate a new Column called 'Power Prediction'
df['Power_Prediction'] = Predict (df.Speed_Mph)
df.sort_values(by =['Speed_Mph'], inplace=True, ascending=True)#Sort the dataset in ascending order

In [ ]:
#Option to view the dataset
df[['Speed_Mph','Power_KWph','Power_Prediction','Cost']].head(5)

In [ ]:
a = np.array(df.Power_KWph)
b = np.array(df.Power_Prediction)
r2 = r2_score(a,b) 
print("\nThe R2 value of Best Fit Line 3 is",(round(r2*100)),"%\n")

*Conclusion: The 'Best Fit' Line is the best approximation as it 'explains' 90% of points on the dataset*

<BR>

### Compare Simple Regression to Other Methods
***

In [ ]:
#https://medium.com/towards-artificial-intelligence/calculating-simple-linear-regression-and-linear-best-fit-an-in-depth-tutorial-with-math-and-python-804a0cb23660#9a2b
#https://colab.research.google.com/drive/1WRdDbFFykiLv16-Vv-ghsregmfdbwtRX?usp=sharing&pli=1#scrollTo=3kbMmi6JYeB4

df2 = df.sample(n=int(len(df))) #Randomly sort the original dataset
num = int(len(df2)*.6) 
#https://datatofish.com/random-rows-pandas-dataframe/
train = df2[:num]#Use the first 60% of the dataset as a training set
test = df2[num:] #Use the remaining 40% of the dataset as a test set
print("\nData",len(df))
print("Train",len(train))
print("Test",len(test),"\n")

In [ ]:
#Main function to find the mean & coefficients of line:

def simple_linear_regression(input_feature,output):
    
    Xi = input_feature
    Yi = output
    
    #Total number of data points:
    n = len(Xi)
    
    #X bar:
    Xi_mean = Xi.mean()
    
    #Y bar:
    Yi_mean = Yi.mean()
    
    #Sum of X:
    S_Xi = (Xi).sum()
    
    #Sum of Y:
    S_Yi = (Yi).sum()
    
    #Sum of (X*Y) multiplied by n:
    S_XiYi = ((Xi*Yi).sum())*n
    
    #Sum of X*Sum of Y:
    S_Xi_S_Yi = S_Xi*S_Yi
    
    #Sum of (X*X) multiplied by n:
    S_XiXi = ((Xi*Xi).sum())*n
    
    #Square of sum of X:
    S_Xi_Square = S_Xi*S_Xi
    
    #Slope:
    slope = (S_XiYi- S_Xi_S_Yi) / (S_XiXi-S_Xi_Square)
    
    #Intercept:
    intercept = Yi_mean - slope * Xi_mean
    
    return slope,intercept

In [ ]:
#Apply df dataset to the function:

Input = np.array((df['Speed_Mph']))
Output = np.array((df['Power_KWph']))

Slope,Intercept = simple_linear_regression(Input,Output)

print ("\nSlope (m)of the Original Dataset  : " , Slope)
print ("Intercept (c) of the Original Dataset :",Intercept,"\n")

In [ ]:
#Training the model with train data:
#Finding the coefficients of best fit line:

actual_slope,actual_intercept = simple_linear_regression(train["Speed_Mph"],train["Power_KWph"])
print ('\nBest Fit Line Values:\n')
print ("Slope of the Training dataset: " ,actual_slope)
print ("Intercept of the Training dataset: " ,actual_intercept,"\n")

#### Plot the regression line with training data

In [ ]:
plt.rcParams["figure.figsize"] = (12,8)

plt.scatter(train["Speed_Mph"],train["Power_KWph"],color="red", label = "Training Dataset")# Scatterplot the training data
plt.plot(train["Speed_Mph"],actual_slope*train["Speed_Mph"]+actual_intercept,color="red", label = "Training Best Fit: 5.57x = -18.45") #Best fit line of the training data
plt.plot (df.Speed_Mph, df.Power_KWph, 'b.', label = "Original Dataset")# Plot the original data
plt.plot(train["Speed_Mph"],Slope * train["Speed_Mph"]+actual_intercept,color="blue",label = "Actual Best Fit: 5.56x = -19.02") #Best fit line of the original data

plt.xlabel("Speed_Mph")
plt.ylabel("Power_KWph")
plt.legend()
plt.show()

#### Define the prediction function

In [ ]:
def get_regression_prediction(input_features,slope,intercept):
    predicted_value = actual_slope*input_features + actual_intercept
    return predicted_value

#### Predicting values based on prediction function

In [ ]:
Speed = 6

estimated_power = get_regression_prediction(Speed,actual_slope,actual_intercept)

print ("\nEstimated Power: ",round(estimated_power,2),"\n")

#### Predicting values for the whole dataset

In [ ]:
Power_Prediction = get_regression_prediction(df["Speed_Mph"],actual_slope,actual_intercept)

#Option to print the prediction
#print(Power_Prediction)

#### Plot the regression line for test data

In [ ]:
plt.scatter(test["Speed_Mph"],test["Power_KWph"],label="Test Dataset")
plt.plot(test["Speed_Mph"],actual_slope*test["Speed_Mph"]+actual_intercept,color="red",label="Test DataBest Fit Line")
plt.xlabel("Speed_Mph")
plt.ylabel("Power_KWph")
plt.legend()
plt.show()

### Using Sklearn package to Model the Data 
***

In [ ]:
#https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html
#https://stackabuse.com/linear-regression-in-python-with-scikit-learn/

X = df[['Speed_Mph']]
Y = df['Power_KWph']

regr = linear_model.LinearRegression()
train_x = np.array(train[['Speed_Mph']])
train_y = np.array(train['Power_KWph'])
regr.fit(train_x,train_y)
test_x = np.array(test[['Speed_Mph']])
test_y = np.array(test['Power_KWph'])

#### Print the Coefficient Values

In [ ]:
coeff_data = pd.DataFrame(regr.coef_ , X.columns , columns=["Coefficients"])
coeff_data

#### Predict the Data

In [ ]:
Y_pred = regr.predict(test_x)

#### Check the Accuracy

In [ ]:
# Check accuracy:
from sklearn.metrics import r2_score
R = r2_score(test_y , Y_pred)
#print ("R² :",R)
print("\nThe R2 value of Test Best Fit Line is",(round(R*100)),"%\n")

### References
***

https://realpython.com/linear-regression-in-python/

https://towardsdatascience.com/linear-regression-detailed-view-ea73175f6e86

https://machinelearningmastery.com/a-gentle-introduction-to-scikit-learn-a-python-machine-learning-library/

https://datatofish.com/sort-pandas-dataframe/

https://www.kite.com/python/answers/how-to-print-an-entire-pandas-dataframe-in-python

https://www.codegrepper.com/code-examples/python/how+to+align+column+name+to+center+in+pandas+dataframe

https://www.dataquest.io/blog/settingwithcopywarning/

https://realpython.com/pandas-groupby/

https://github.com/ianmcloughlin/jupyter-teaching-notebooks/blob/master/simple-linear-regression.ipynb

https://www.geeksforgeeks.org/python-coefficient-of-determination-r2-score/

https://medium.com/towards-artificial-intelligence/calculating-simple-linear-regression-and-linear-best-fit-an-in-depth-tutorial-with-math-and-python-804a0cb23660#9a2b

https://colab.research.google.com/drive/1WRdDbFFykiLv16-Vv-ghsregmfdbwtRX?usp=sharing&pli=1#scrollTo=3kbMmi6JYeB4

https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/

https://datatofish.com/random-rows-pandas-dataframe/

https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html

https://contactsunny.medium.com/linear-regression-in-python-using-scikit-learn-f0f7b125a204

https://stackabuse.com/linear-regression-in-python-with-scikit-learn/

https://www.geeksforgeeks.org/linear-regression-python-implementation/

# End